In [ ]:
def create_sentences(df, cols_name):
    # В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    # Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    # Уровень осадков повысится / понизится на X единиц за Y дней.
    dates = [i.split()[0] for i in cols_name]
    days = [i.split('.')[0] for i in dates]
    months = (months_names[int(i.split('.')[1]) - 1] for i in dates)
#     months_words = (morph.parse(i)[0] for i in months_names)
    dates = [day + ' ' + month for day, month in zip(days, months)]
    cols = [df[i].apply(lambda x: x.replace('−', '-').replace(',', '.')) for i in cols_name]
    out = []
#     print(dates)
    changes = {i : float(cols[1][i]) - float(cols[0][i]) for i in df.index}
    
    create_temp_str = lambda x, s: f'''{dates[1]} {s} температура {'изменится' if x == 0 else ('повысится' if x > 0 else 'понизится')} на {abs(int(x))} {morph.parse('градус')[0].make_agree_with_number(abs(int(x))).word} по сравнению с ''' + dates[0]
    create_precip = lambda x: f'''Уровень осадков {'не изменится' if x == 0 else ('повысится' if x > 0 else 'понизится')} на {abs(x)} {morph.parse('единица')[0].make_agree_with_number(x).word} '''
    print(create_temp_str(changes['maxTemp'], 'максимальная'))
    print(create_temp_str(changes['minTemp'], 'минимальная'))
    
    print(f'''Скорость ветра изменится на {int(changes['windSpeed'])} единиц в {dates[1]} по сравнению с {dates[0]}.''')
    print(create_precip(changes['precipitation']))

    
create_sentences(df, (df.columns[0], df.columns[2]))

# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 [10 баллов] до 23:59 17.02.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе с gismeteo.ru
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных [3 балла]

Выберите произвольным образом город в России и найдите прогноз погоды в нем на ближайшие 10 дней на сайте gismeteo.ru.

Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание.  

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

Сделаем import всех необходимых библиотек.

In [1]:
from scrapy.crawler import CrawlerProcess
from twisted.internet.error import ReactorNotRestartable
import scrapy

In [2]:
import random
import pandas as pd
from json import dump
import datetime
import re

Выберем случайный российский город. Данные взяты из [Википедии](https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia).

In [3]:
CITIES_URL = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia'
CITY_SELECTOR = '//tr/td[2]'

In [4]:
cities = []
class RussianCities(scrapy.Spider):
    name = 'russian_cities'
    start_urls = [CITIES_URL,]
    def parse(self, response):
        for city in response.xpath(CITY_SELECTOR):
            cities.append(city.xpath('text()').extract_first())

In [5]:
process = CrawlerProcess()
process.crawl(RussianCities)
process.start()

2018-02-16 11:50:24 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-02-16 11:50:24 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.2 (default, Jul 17 2017, 16:44:47) - [GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Darwin-15.6.0-x86_64-i386-64bit
2018-02-16 11:50:24 [scrapy.crawler] INFO: Overridden settings: {}
2018-02-16 11:50:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-02-16 11:50:24 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaulthea

In [6]:
len(cities)

1112

In [7]:
random.seed(4118956)
print(random.choice(cities))

Обнинск


Выбрали Сергиев Посад. 

In [8]:
CITY_URL = 'https://www.gismeteo.ru/weather-sergiyev-posad-11958/10-days/'

SELECTORS = {
    'weekday': '//div[@class="forecast_frame"]//div[@class="w_date__day"]/text()',
    'day': '//span[contains(@class, "w_date__date")]/text()',
    'min_temp': '//div[@class="maxt"]/text()',
    'max_temp': '//div[@class="mint"]/text()',
    'wind_speed': '//div[contains(@class,"w_wind__warning")]/text()',
    'precipitation': '//div[contains(@class,"w_precipitation__value")]/text()',
}

def process_digit(d):
    '''
    Removes all junk like \n and whitespaces.
    Returns: pure float number.
    '''
    return re.findall('[\d,]+', d)[0]

Данные о погоде будут собираться по состоянию на 12.02.2018

In [9]:
json_data = []
class GismeteoForecast(scrapy.Spider):
    name = 'gismeteo_forecast'
    start_urls = [CITY_URL,]
    def parse(self, response):
        weekdays = response.xpath(SELECTORS['weekday']).extract()
        days = response.xpath(SELECTORS['day']).extract()[:10]
        min_temp = response.xpath(SELECTORS['min_temp']).extract()[:10]
        max_temp = response.xpath(SELECTORS['max_temp']).extract()[:10]
        wind_speed =  response.xpath(SELECTORS['wind_speed']).extract()[:10]
        precipitation = response.xpath(SELECTORS['precipitation']).extract()
        for i in range(10):
            json_data.append({
                'weekday': weekdays[i],
                'day': process_digit(days[i]),
                'minTemp': min_temp[i],
                'maxTemp': max_temp[i],
                'windSpeed': process_digit(wind_speed[i]),
                'precipitation': process_digit(precipitation[i]),
            })

In [16]:
# try:
#     try:
#         del process
#     except:
#         pass
#     if 
#     process = CrawlerProcess()
process.crawl(GismeteoForecast)
process.start()
# except ReactorNotRestartable as ex:
# #     print('\n\n\n------------\nRestart kernel and rerun this cell.\n------------')
#     print(ex)

2018-02-16 11:52:51 [scrapy.crawler] INFO: Overridden settings: {}
2018-02-16 11:52:51 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-02-16 11:52:51 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.http

ReactorNotRestartable: 

Сохраним результаты в json.

In [10]:
with open('result.json', 'w') as fp:
    dump(json_data, fp)

Считаем в dataframe записанный json-файл.

In [11]:
df = pd.read_json('result.json')

In [12]:
df

,day,maxTemp,minTemp,precipitation,weekday,windSpeed
0,16,−8,−5,"0,6",Пт,9
1,17,−6,−5,"1,4",Сб,10
2,18,−6,−4,"1,7",Вс,8
3,19,−6,−4,"3,5",Пн,5
4,20,−10,−7,"0,6",Вт,8
5,21,−15,−10,"0,1",Ср,7
6,22,−16,−9,"0,6",Чт,7
7,23,−16,−11,"0,4",Пт,10
8,24,−15,−13,"0,2",Сб,9
9,25,−17,−12,"0,2",Вс,12


Зададим таблице правильный индекс и транспонируем.

In [13]:
days = df['day']
weekdays = df['weekday']
new_index = []

for day, w_day in zip(days, weekdays):
    new_index.append('{d}.0{m} ({w})'.format(d=day,
                                            w=w_day.encode('utf-8'),
                                            m=datetime.date.today().month))

df.drop(['day', 'weekday'], axis=1, inplace=True)
df.index = new_index
df = df.T

In [14]:
df

,16.02 (b'\xd0\x9f\xd1\x82'),17.02 (b'\xd0\xa1\xd0\xb1'),18.02 (b'\xd0\x92\xd1\x81'),19.02 (b'\xd0\x9f\xd0\xbd'),20.02 (b'\xd0\x92\xd1\x82'),21.02 (b'\xd0\xa1\xd1\x80'),22.02 (b'\xd0\xa7\xd1\x82'),23.02 (b'\xd0\x9f\xd1\x82'),24.02 (b'\xd0\xa1\xd0\xb1'),25.02 (b'\xd0\x92\xd1\x81')
maxTemp,−8,−6,−6,−6,−10,−15,−16,−16,−15,−17
minTemp,−5,−5,−4,−4,−7,−10,−9,−11,−13,−12
precipitation,"0,6","1,4","1,7","3,5","0,6","0,1","0,6","0,4","0,2","0,2"
windSpeed,9,10,8,5,8,7,7,10,9,12


### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на ближайшие 10 дней в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

Некоторые вспомогательные функции, которые вам понадобятся: согласование существительного с числительным и приведение существительного к нужному падежу: 

In [24]:
import pymorphy2

In [3]:
df = pd.read_csv('/Users/Air_Folder/Downloads/gismeteo_forecast.csv', sep=';', index_col=0)

In [4]:
df.head()

,13.02 (Вт),14.02 (Ср),15.02 (Чт),16.02 (Пт),17.02 (Сб),18.02 (Вс),19.02 (Пн),20.02 (Вт),21.02 (Ср),22.02 (Чт)
maxTemp,−13,−14,−13,−8,−5,−4,−3,−2,−4,−9
minTemp,−7,−8,−8,−4,−2,−1,−1,+3,+3,−4
precipitation,0,0,"0,2",3,"3,8","0,8","0,8","8,8","0,2",0
windSpeed,7,8,7,10,10,8,9,15,15,5


In [5]:
name_of_city = 'SOME_CITY_NAME'

In [6]:
months_names = ('января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 
                'июля', 'августа', 'сентября', 'октября', 'ноября',  'декабря')

In [125]:
def create_sentences(df, cols_name):
    # В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    # Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    # Уровень осадков повысится / понизится на X единиц за Y дней.
    dates = [i.split()[0] for i in cols_name]
    days = [i.split('.')[0] for i in dates]
    months = (months_names[int(i.split('.')[1]) - 1] for i in dates)
    dates = [day + ' ' + month for day, month in zip(days, months)]
    cols = [df[i].apply(lambda x: x.replace('−', '-').replace(',', '.')) for i in cols_name]

    changes = {i : float(cols[1][i]) - float(cols[0][i]) for i in df.index}
    
    def get_changing_str(x, units):
        changing_str = None
        if x == 0:
            changing_str = 'не изменится'
        else:
            if x > 0:
                changing_str = 'повысится'
            else:
                changing_str = 'понизится'
            changing_str += f" на {abs(x)} {morph.parse(units)[0].make_agree_with_number(abs(x)).word}"
            
        return changing_str
        
    create_temp_str = lambda x, s: f'''{dates[1]} {s} температура {get_changing_str(int(x), 'градус')} по сравнению с ''' + dates[0]
    create_precip = lambda x: f'''В период с {dates[0]} по {dates[1]} уровень осадков {get_changing_str(round(x, 1), '')}'''
    
    print(create_temp_str(changes['maxTemp'], 'максимальная'))
    print(create_temp_str(changes['minTemp'], 'минимальная'))
    print(f'''Скорость ветра изменится на {int(changes['windSpeed'])} {morph.parse('единица')[0].inflect({'accs'}).make_agree_with_number(abs(int(changes['windSpeed']))).word} {dates[1]} по сравнению с {dates[0]}''')
    print(create_precip(changes['precipitation']))

    
create_sentences(df, (df.columns[0], df.columns[1]))

14 февраля максимальная температура понизится на 1 градус по сравнению с 13 февраля
14 февраля минимальная температура понизится на 1 градус по сравнению с 13 февраля
Скорость ветра изменится на 1 единицу 14 февраля по сравнению с 13 февраля
В период с 13 февраля по 14 февраля уровень осадков не изменится


In [126]:
for ind_i, i in enumerate(df.columns[:-1]):
    for j in df.columns[ind_i + 1:]:
        create_sentences(df, (i, j))

14 февраля максимальная температура понизится на 1 градус по сравнению с 13 февраля
14 февраля минимальная температура понизится на 1 градус по сравнению с 13 февраля
Скорость ветра изменится на 1 единицу 14 февраля по сравнению с 13 февраля
В период с 13 февраля по 14 февраля уровень осадков не изменится
15 февраля максимальная температура не изменится по сравнению с 13 февраля
15 февраля минимальная температура понизится на 1 градус по сравнению с 13 февраля
Скорость ветра изменится на 0 единиц 15 февраля по сравнению с 13 февраля
В период с 13 февраля по 15 февраля уровень осадков повысится на 0.2 
16 февраля максимальная температура повысится на 5 градусов по сравнению с 13 февраля
16 февраля минимальная температура повысится на 3 градуса по сравнению с 13 февраля
Скорость ветра изменится на 3 единицы 16 февраля по сравнению с 13 февраля
В период с 13 февраля по 16 февраля уровень осадков повысится на 3.0 
17 февраля максимальная температура повысится на 8 градусов по сравнению с 1

### 3. Ответьте на вопросы [3 балла]
**Вопрос:** В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам?

**Ответ:** 
1. Генерация текста по шаблонам может понадобиться в следующих случаях:
1. Автоматические новости на новостных сайтах
1. Автоматические описания товаров на сайтах интернет магазинов
1. Реклама однотипных товаров
1. Автоматическая служба поддержки
1. Бот, общающийся с пользователем на естественном языке
1. Описание изменений курса валют
1. Бегущая строка в метро
1. Поздравления с праздниками в корпоративных помещениях

**Вопрос:** В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?

**Ответ:**
1. Описание изменений курса валют
1. Описание состояния показателей производства (производительность, динамика эффективности)
1. Генерация отчетов по работе предприятия
1. Генерация отчетов по экономических показателям

**Вопрос:** Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

**Ответ:** Можно добавить новые шаблоны. Или например, динамически формировать полностью предложения вместо некоторых подстановок. Знание синтаксической структуры позволяет достаточно легко осуществлять подстановки, согласованные с остальными словами предложения.

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 17.02.2018.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 1: Анна Лапидус (email: anyalapidus@list.ru, telegram: @alpids).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

### Как сдать домашнее задание в AnyTask
* Зарегистрируйтесь в системе AnyTask по ссылке http://anytask.org/accounts/register . Регистрация обязательна для всех!
* Подтвердите регистрацию по e-mail.
* Зайдите в свой профиль, нажмите “Активация инвайтов на курсы” и введите инвайт:

ИАД-1 : VNYQlGd

ИАД-2 : wDEa8ge

ИАД-3 : SviK5dd

ИАД-4 : dzdD8YG


 У вас появится курс “ВШЭ > Прикладные задачи анализа данных” в разделе “Посещает курсы”.
* Перейдите по ссылке “ВШЭ > Прикладные задачи анализа данных (2018)” и нажмите кнопку “Сдать”. 
* У вас откроется условие задачи и будут доступны различные поля, в частности, НИЖЕ условия задачи будет поле ввода, в которое вы сможете вписать какой-то комментарий, и сможете прикрепить файл. Сделайте это.
* Домашнее задание лучше всего сдавать в форматах IPYNB.
* Оценку вы получите также в системе AnyTask. За своей успеваемостью можете следить в разделе “Ведомость”, а также можете прокомментировать что-то в каждом вашем домашнем задании, зайдя на ее страничку (ячейки в табличке на страничке “Ведомость” кликабельны и ведут на ваш submission домашки).

(**ВАЖНО**) Если домашнее задание вы делали в группе, то в AnyTask домашнее сдает *один* участник группы, но заргестрироваться в AnyTask обязательно всем – так мы сможем проставить вам оценки в ведомость в AnyTask.

Ссылка на курс в AnyTask: http://anytask.org/course/281